In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME")
TEAM_TRACK = os.getenv("TEAM_TRACK")
HOME = os.getenv("HOME")

In [ ]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
from word2number.w2n import word_to_num

In [ ]:
input_dir = Path(f"{HOME}/{TEAM_TRACK}")

with open(input_dir / "nlp.jsonl", "r") as f:
    instances = [json.loads(line.strip()) for line in f if line.strip() != ""]

In [ ]:
df = pd.DataFrame(instances)
df = df.set_index("key")
display(df)

In [ ]:
display(df.tool.unique())

# Case 1: Prompt model to use exact wording, electromagnetic pulse, EMP, surface-to-air with hyphens
# Case 2: Always lower case the tool, unless its an acronym
# Anti-case: There is a fixed set of tools

In [ ]:
tools = df.tool.unique()
for tool in tools:
    rows = df[df.tool == tool].transcript
    for row in rows:
        for other in tools:
            if tool == other:
                continue
            if other.lower() in row.lower():
                print("Real: ", tool, " | ", row)

# Case: always use the first appearance (unless its a bracket)

In [ ]:
a = (
    df.transcript.str.lower()
    .replace("[^a-zA-Z0-9]", " ", regex=True)
    .str.split()
    .explode()
    .unique()
)
b = []

for w in a:
    if len(w) < 3:
        continue
    # try:
    #    word_to_num(w)
    # except:
    #    b.append(w)

# display(b)

df.transcript.str.lower().str.split("").explode().unique()
df[df.transcript.str.contains("1")]

In [ ]:
# fmt: off
rm_list = (
    ",", "and", "red", "blue", "orange", "purple", "cyan", "gray", "grey",
    "brown", "yellow", "silver", "black", "green", "white", "camouflage"
)
# fmt: on

In [ ]:
# Note: From looking at eval code, it is both punctuation & case sensitive.
display(len(df.target.unique()))
display(sum(df.target.str.isupper()))


def rm(s):
    for r in rm_list:
        s = s.replace(r, "")
    return " ".join(s.split())


targets = set(rm(t) for t in df.target.unique())
display(targets)

df.transcript.str.extractall(r"\((.*?)\)")

# Case 1: Preserve punctuation
# Case 2: Always lower case target
# Case 3: Preserve order
# Case 4: lmao rainbow target
# Case 5: Bracketed words

In [ ]:
a = set(df.target.str.replace(",", " ").str.split().explode())
b = set(df.target.str.split().str[-2:].str.join(" "))
display(b)
display(a - b - {"and"})

In [ ]:
display(sorted(df.heading.unique()))

# Anti-case: always multiple of five
# Anti-case: no 000
# Anti-case: always 0 to 360
# Is there negative heading?

In [ ]:
df.loc[1].transcript